# Import Depedencies

In [16]:
import keras
from keras import backend as K
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
import numpy as np
from IPython.display import Image
from keras.optimizers import Adam

## Import MobileNet and insert Layer - Activation

In [106]:
base_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(2,activation='softmax')(x) #final layer with softmax activation

In [107]:
model=Model(inputs=base_model.input,outputs=preds)

Arsitektur MobileNet

In [108]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, None, None, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 32)    128       
__________

## Setting Layer

In [109]:
for layer in model.layers:
    layer.trainable=False
# or if we want to set the first 20 layers of the network to be non-trainable
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

## Input Image dan Setting CNN

In [110]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

train_generator=train_datagen.flow_from_directory('c:/Users/farvn/Downloads/Dataset',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=16,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 44 images belonging to 2 classes.


## Compile Model

In [111]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# optimizer = adam
# loss function = categorical cross entropy
# evaluation metric = accuracy

In [112]:
step_size_train=train_generator.n//train_generator.batch_size

## Train Model

In [117]:
process = model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=20)

Epoch 1/20
2/2 [==============================] - 5s 2s/step - loss: 2.2911e-07 - acc: 1.0000
Epoch 2/20
2/2 [==============================] - 5s 3s/step - loss: 1.1921e-07 - acc: 1.0000
Epoch 3/20
2/2 [==============================] - 3s 2s/step - loss: 3.9328e-05 - acc: 1.0000
Epoch 4/20
2/2 [==============================] - 4s 2s/step - loss: 9.8907e-07 - acc: 1.0000
Epoch 5/20
2/2 [==============================] - 4s 2s/step - loss: 1.1921e-07 - acc: 1.0000
Epoch 6/20
2/2 [==============================] - 4s 2s/step - loss: 9.4364e-05 - acc: 1.0000
Epoch 7/20
2/2 [==============================] - 4s 2s/step - loss: 1.3039e-07 - acc: 1.0000
Epoch 8/20
2/2 [==============================] - 3s 2s/step - loss: 2.1341e-07 - acc: 1.0000
Epoch 9/20
2/2 [==============================] - 3s 2s/step - loss: 1.2701e-07 - acc: 1.0000
Epoch 10/20
2/2 [==============================] - 4s 2s/step - loss: 1.9912e-06 - acc: 1.0000
Epoch 11/20
2/2 [==============================] - 4s 2s/st

## Predict

In [82]:
def load_image(img_path, show=False):

    img = image.load_img(img_path, target_size=(150, 150))
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 255.                                      # imshow expects values in the range [0, 1]

    if show:
        plt.imshow(img_tensor[0])                           
        plt.axis('off')
        plt.show()

    return img_tensor

In [83]:
img_path = 'C:/Users/farvn/Downloads/test.jpg'
new_image = load_image(img_path)

pred = model.predict(new_image)

pred

array([[7.4115807e-05, 4.4776294e-02, 4.1717555e-02, 9.1343206e-01]],
      dtype=float32)

In [85]:
img_path = 'C:/Users/farvn/Downloads/ph.jpg'
new_image = load_image(img_path)

pred = model.predict(new_image)
pred


array([[6.4634722e-01, 3.5339960e-01, 2.2711034e-04, 2.6050386e-05]],
      dtype=float32)